<a href="https://colab.research.google.com/github/Rustydinoco/ML_Learning/blob/master/Convolution_Neural_Network_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [35]:
# Convert MNIST Image File into a Tensor of 4-Dimensions (Images, Height, Width, Color channel)
transform =  transforms.ToTensor()

In [36]:
# Train Data
train_data = datasets.MNIST(root= '/cnn_data', train=True, download=True, transform=transform)

In [37]:
#Test data
test_data = datasets.MNIST(root= '/cnn_data', train=False, download=True, transform=transform)

In [38]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [39]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [40]:
# Build The Model
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)
    # Fully Connected Layers
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, 2, 2) #2x2 kernel and stride 2
    # Second Pass
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, 2, 2)

    # Re-view to flatten it out
    x = x.view(-1, 16*5*5)

    # Fully Connected Layers
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return F.log_softmax(x, dim=1)


In [41]:
# Create an Instance of our Model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [42]:
# Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [33]:
import time
start_time = time.time()
train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10, shuffle = False)
# Create Variables To Tracks Things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For Loop of Epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0

  #Train
  for b, (X_Train, y_train) in enumerate(train_loader):
    b+=1 # Start our batches at 1
    y_pred = model(X_Train) # Get Predicted values from the training set (2D)
    loss = criterion(y_pred, y_train) # Compare the predicted values

    predicted = torch.max(y_pred.data, 1)[1] # add up the number of correct predictions.Indexed off the first point
    batch_corr = (predicted == y_train).sum() # How many we got correct from this batch. True = 1, False = 0, sum those up
    trn_corr += batch_corr # keep track as we go along in training.

    # Update Our Parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the results
    if b%600 == 0:
      print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')

  train_losses.append(loss.item())
  train_correct.append(trn_corr)

  #  Test
  with torch.no_grad(): # No gradient so we don't update our weight and bias test data
    for b, (X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted  = torch.max(y_val.data, 1)[1] # Adding up correct predictions
      tst_corr += (predicted == y_test).sum() # T=1 F=0 and sum away

  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)


current_time = time.time()
total = current_time - start_time
print(f'Training Took: {total/60} minutes! ')


Epoch: 0  Batch: 600  Loss: 0.00036099221324548125
Epoch: 0  Batch: 1200  Loss: 0.0001712304074317217
Epoch: 0  Batch: 1800  Loss: 0.0005378525820560753
Epoch: 0  Batch: 2400  Loss: 0.027378559112548828
Epoch: 0  Batch: 3000  Loss: 0.0002596829435788095
Epoch: 0  Batch: 3600  Loss: 8.657989383209497e-05
Epoch: 0  Batch: 4200  Loss: 0.00040106248343363404
Epoch: 0  Batch: 4800  Loss: 1.2349883036222309e-05
Epoch: 0  Batch: 5400  Loss: 0.00046615247265435755
Epoch: 0  Batch: 6000  Loss: 0.000162848416948691
Epoch: 1  Batch: 600  Loss: 6.399912672350183e-05
Epoch: 1  Batch: 1200  Loss: 0.0002539866545703262
Epoch: 1  Batch: 1800  Loss: 0.0003175207821186632
Epoch: 1  Batch: 2400  Loss: 8.683211490279064e-05
Epoch: 1  Batch: 3000  Loss: 0.00015392867499031126
Epoch: 1  Batch: 3600  Loss: 0.12387670576572418
Epoch: 1  Batch: 4200  Loss: 0.00015394852380268276
Epoch: 1  Batch: 4800  Loss: 0.00032526275026611984
Epoch: 1  Batch: 5400  Loss: 0.08193918317556381
Epoch: 1  Batch: 6000  Loss: 1.2

In [43]:
train_losses = [tl.item() for tl in train_losses]
plt.plot(train_losses, label="Training Loss")
plt.plot(test_losses, label= "Validation Loss")
plt.title("Loss at epochs")
plt.legend()


AttributeError: 'float' object has no attribute 'item'